# Regression with Statsmodels

In [1]:
import pandas as pd 
import numpy as np
import statsmodels
import statsmodels.formula.api as smf
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer

from statsmodels.stats.anova import anova_lm

%matplotlib inline

In [2]:
d = pd.read_csv('/Users/douglashughes/Downloads/dataverse_files-2/dVote.csv')
d.columns = ['in_dex', 'vote_center', 'treat', 'age', 'gender', 'education', 'years_in_us', 'ideology']

What is the story with the data in here? Are there any strange things that are happening? Maybe in the `ideology` variable that is a 1-7 likert scale? 

In [3]:
d.head()

,in_dex,vote_center,treat,age,gender,education,years_in_us,ideology
0,1,1.0,1.0,42.0,2,5.0,15.0,8.0
1,2,0.0,3.0,39.0,2,6.0,25.0,2.0
2,3,0.0,4.0,49.0,1,4.0,17.0,8.0
3,4,1.0,3.0,61.0,1,1.0,24.0,6.0
4,5,0.0,3.0,19.0,1,12.0,9.0,2.0


Ok, we're going to have to do some cleaning of this file to get it ready to go.. 

1. We don't need the extra index column; could probably fix this at read, but :shrug: 
2. The treatment probably can be just an int, since it is actually a category indicator. 
3. Age can be an int 
4. Gender is stored as an int; but we're going to need to know what those lables are 
5. Years in us can be an int. 
6. Ideology can can also be an int. 

Also, there is some item-level missingness. What would you like to do with it? **Blatently** drop it you say? Well, ok then. 

In [4]:
d.drop('in_dex', axis=1, inplace=True)
d.dropna(inplace=True)
d = d.astype('int') # no inplace method... because consistency 

In [5]:
d.head()

,vote_center,treat,age,gender,education,years_in_us,ideology
0,1,1,42,2,5,15,8
1,0,3,39,2,6,25,2
2,0,4,49,1,4,17,8
3,1,3,61,1,1,24,6
4,0,3,19,1,12,9,2


Nice nice. 

# But, what is actually happening in this data?? 

We're looking at an experiment where individuals who were born in Mexico but live in the United States were asked about "Which of these candidates do you think that you are most likely to support, if there were an election between them tomorrow." 

So, we're building an information environment where we can manpulate the features that are available to them to base their decision. 

Here's what that information environment looks when we place subjects into the 'white' condition: 

![white](./img/faces1.jpg)

What it looks like when we put them into the 'mestizo' condition: 

![mestizo](./img/faces2.jpg)

And, here's what the information environment looks like when we place them into the 'indigenous' condition: 

![img](./img/faces3.jpg)

We also provide people with issue statements that are basically, and intentionally, not that interesting. 

What if we re-level those treatment conditions so that they make a bit more sense. I'm going to subtract one from each of the conditions, so that the control condition (where they get no images) is scored 0, and the other treatment conditions range {1:3}. 

In [6]:
d['treat'] = d['treat'] - 1

In [7]:
d.head()

,vote_center,treat,age,gender,education,years_in_us,ideology
0,1,0,42,2,5,15,8
1,0,2,39,2,6,25,2
2,0,3,49,1,4,17,8
3,1,2,61,1,1,24,6
4,0,2,19,1,12,9,2


# Using statsmodels

Stats models is built for *rapid* development and testing of linear models in python. They leverage some of the great parts of R -- namely the *formulas* style interface for writing out dependend and independent variable concepts. (Yes, you could get this with [patsy](https://patsy.readthedocs.io/en/latest/) formulas, but there is more that we want from statsmodels too. 

Formulas are structured as: 

`DV ~ IV` 

Where the outcome that we're interested in is the IV, and those model features that we've designed are the IVs. 

In general, we are going to: 

- Specify a model
- Fit that model (with optional args for the vcov structure) 
- Inspect features of that model. 

If we wanted to know, "What are the chances that someone votes for the *experimental candidate* and examine if those rates are different in across treatments, how would we structure this? 

In [8]:
mod1 = smf.ols('vote_center ~ treat', data = d)
fit_mod1 = mod1.fit()

In [9]:
fit_mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            vote_center   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.900
Date:                Tue, 02 Oct 2018   Prob (F-statistic):              0.170
Time:                        18:34:15   Log-Likelihood:                -108.66
No. Observations:                 164   AIC:                             221.3
Df Residuals:                     162   BIC:                             227.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2685      0.061      4.404      0.000       0.148       0.389
treat          0.0446      0.032      1.378      0.170      -0.019       0.109
==============================================================================
Omnibus:                      841.733   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.796
Skew:                           0.688   Prob(JB):                     9.21e-07
Kurtosis:                       1.525   Cond. No.                         3.72
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

With this output, we can read into the performance of our model, and interpret coefficeints. In this case, we can know that for each marginal change in the treatment the person was in, there is a 9% increase in the likelihood of voting for the experimental candidate. 

*Does that make sense...*

## Any Treatment? 
What if people received any form of picture? Did this affect their probability of voting for the center candidate? There are two ways, practically, that we can undertake this question. First, we could include a new feature onto the dataset that is just a `True` if they were in any of the treatment conditions, and `False` if they were in the control condition. This is pretty nice, and makes a persistent feature that we could refer to later. 

In [10]:
d['any_treat'] = d['treat'] > 0

In [11]:
d.head()

,vote_center,treat,age,gender,education,years_in_us,ideology,any_treat
0,1,0,42,2,5,15,8,False
1,0,2,39,2,6,25,2,True
2,0,3,49,1,4,17,8,True
3,1,2,61,1,1,24,6,True
4,0,2,19,1,12,9,2,True


The other way would be to make a more ephemeral feature, within the model call, that will *not* persist beyond the model call. On the one hand, this is nice because it keeps from making a large number of transforms against the data; and, also, it doesn't engineer new data that we might not end up using. On the other hand, though, it does increase the time that it takes for a model to run, because we have to compute the vector transform at the time of the model fit. 

In [12]:
any_treat1 = smf.ols('vote_center ~ any_treat', data = d).fit()
any_treat2 = smf.ols('vote_center ~ C(treat > 0)', data = d).fit()

In [13]:
any_treat1.params

Intercept            0.302326
any_treat[T.True]    0.044782
dtype: float64

In [14]:
any_treat2.params

Intercept               0.302326
C(treat > 0)[T.True]    0.044782
dtype: float64

The good news, it seems, is that these produce identical estimates. 

## Factors of Treatment 
Of course, the *marginal* interpretation of our treatment conditions was incorrect. Those are actually categorical variables that we assigned to people, not a numeric feature that varries with consistent difference between levels. 

Just like when creating the `any_treat` feature, we could build the new version in a number of ways. If we want to make a permanent feature, we can use `pd.get_dummies()`. 

In [15]:
d.head()

,vote_center,treat,age,gender,education,years_in_us,ideology,any_treat
0,1,0,42,2,5,15,8,False
1,0,2,39,2,6,25,2,True
2,0,3,49,1,4,17,8,True
3,1,2,61,1,1,24,6,True
4,0,2,19,1,12,9,2,True


In [16]:
dummies = pd.get_dummies(d['treat'], prefix='treat')
d = pd.concat([d, dummies], axis=1)

mod_dummies_1 = smf.ols('vote_center ~ 1 + treat_1 + treat_2 + treat_3', data = d).fit()

But, we can also do this work using the `C` operator in the formula notation. 

In [17]:
mod_dummies_2 = smf.ols('vote_center ~ 1 + C(treat)', data = d).fit()

In [18]:
mod_dummies_1.params

Intercept    0.302326
treat_1      0.005367
treat_2     -0.045915
treat_3      0.162791
dtype: float64

In [19]:
mod_dummies_1.params

Intercept    0.302326
treat_1      0.005367
treat_2     -0.045915
treat_3      0.162791
dtype: float64

Wait, aren't those numeric variables in the first call? Aren't they factor variables in the second? **Yes**. What about the dummy variables being coded as 0/1 and as True/False makes this possible? 

# Can we improve this with covariates? 

One of the claims that we made before is that covariates that are predictive of outcomes should improve the fit of the model, without changing measurably the cofficient that we estimate for the model. Let's see if this works. 

In [20]:
d.head()

,vote_center,treat,age,gender,education,years_in_us,ideology,any_treat,treat_0,treat_1,treat_2,treat_3
0,1,0,42,2,5,15,8,False,1,0,0,0
1,0,2,39,2,6,25,2,True,0,0,1,0
2,0,3,49,1,4,17,8,True,0,0,0,1
3,1,2,61,1,1,24,6,True,0,0,1,0
4,0,2,19,1,12,9,2,True,0,0,1,0


In [21]:
mod3 = smf.ols('vote_center ~ 1 + C(treat)', data = d).fit()
mod4 = smf.ols('vote_center ~ 1 + C(treat) + C(gender) + C(ideology)', data = d).fit()

Let's look to see what, if anything changes beween the two models. 

In [22]:
sg = Stargazer([mod3, mod4]).render_html()
HTML(sg)

There are a *lot* of levels in that ideology feature; rather than understanding whether any one level of those features is useful in explaining the outcome, what if we wanted to know instead whether *all* the levels of that entire feature were useful? 

Enter the f-test. Here, we can use the `statsmodels.stats.anova.anova_lm` method to look at whether restricting *all* of the levels of a feature to be zero (that is, requiring that there is no effect of that parameter) makes the model predict the outcome measurably (i.e. significantly) worse than letting those parameters be free. 

In [23]:
statsmodels.stats.anova.anova_lm(mod4)
# yep, there's a huge error that gets thrown; what I read about it suggests that this is not a concern for us. 

/usr/local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(treat),3.0,1.043846,0.347949,1.632977,0.184128
C(gender),1.0,0.576797,0.576797,2.706995,0.101989
C(ideology),8.0,2.759707,0.344963,1.618966,0.123721
Residual,151.0,32.174528,0.213076,NaN,NaN


As we read this, there isn't a lot to suggest that *any* of these features, on their own, are predictive of outcomes. One thing to keep in mind is that: 

- Features that we *design* to be causal, retain that in these estimates. 
- Features that we **do not** *design* to be causal don't pick up any causal flavor just because we've put them through a regression. 

# Fitting with Heteroskedastic Consistent (Robust) Standard Errors 
David gives a great justification for why "robust" standard errors are well-justified in our data. And, implementing these with `statsmodels` borders on being trivial. 

If you have a model that has already been fit, you use a function against the model object to pull the `HC1` errors from that model object. 

In [24]:
statsmodels.regression.linear_model.RegressionResults.HC1_se(mod3)

array([ 0.0709074 ,  0.10308444,  0.10019432,  0.1046812 ])

However, more common in practice is to call for the robust standard errors at the time that you're fitting the model. On the one hand, this does remove your ability to pull off *standard* standard errors, but on the other hand, you probably want robust SEs anyways. 

In [25]:
robust_mod3 = smf.ols('vote_center ~ 1 + C(treat)', data = d).fit(cov_type = 'HC1')

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [27]:
sg = Stargazer([mod3, robust_mod3]).render_html()
HTML(sg)

As you can see in this set, there is very little change in the estimates of the SEs; because the data is relatively well behaved across the parameter set. 